In [1]:
import sys
import os

# Insert the parent directory of "app" into sys.path
# so that Python recognizes "app" as an importable package.
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(parent_dir)

In [2]:
from dotenv import load_dotenv
load_dotenv()  # This looks for .env in the current working directory

True

In [3]:
import time

import pandas as pd
import seaborn as sns
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    context_recall,
    faithfulness
)

from langchain_core.messages import HumanMessage, ToolMessage, AIMessage, ToolMessage


In [4]:
# Benchmark data
test_df = pd.read_csv('benchmark.csv', delimiter=";")
test_df.head()

,question,ground_truth
0,What are some contracts that ended after 1990-...,I found 323 contracts that ended after 1990-01...
1,What are some contracts that ended after 1990-...,"There are 52 contracts classified as ""Strategi..."
2,"What are some contracts with HC2 Holdings, Inc.?","There is one contract involving HC2 Holdings, ..."
3,Which year was the most contracts signed?,The year with the most contracts signed was 20...
4,Which company has the most active contracts?,NaN


In [5]:
from backend.agent import get_agent

In [6]:
agent = get_agent()

In [7]:
input_messages = [HumanMessage(content="Which day is today?")]
messages = agent.invoke({"messages": input_messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Which day is today?
================================== Ai Message ==================================

Today is March 24, 2025.


In [8]:
# History
messages

{'messages': [HumanMessage(content='Which day is today?', additional_kwargs={}, response_metadata={}, id='cec2978f-cccd-40f0-aeb8-6f6a9f127a89'),
  AIMessage(content='Today is March 24, 2025.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 792, 'total_tokens': 804, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90d33c15d4', 'finish_reason': 'stop', 'logprobs': None}, id='run-f32b685e-8774-4c14-aa5d-f2eaccee8a9b-0', usage_metadata={'input_tokens': 792, 'output_tokens': 12, 'total_tokens': 804, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [9]:
# Chat with history
input_messages = [HumanMessage(content="List all contracts where a party from Japan is involved.")]
messages = agent.invoke({"messages": input_messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

List all contracts where a party from Japan is involved.
================================== Ai Message ==================================
Tool Calls:
  ContractSearch (call_XGilOOShS13EfD9F7WlJ5npw)
 Call ID: call_XGilOOShS13EfD9F7WlJ5npw
  Args:
    country: JP
================================= Tool Message =================================
Name: ContractSearch

[{"output": {"total_count_of_contracts": 10, "example_values": [{"summary": "Amendment and termination of joint venture agreement between Veoneer Parties and Nissin Parties. The agreement terminates the JV Agreement with respect to VNBJ and VNBZ upon the VNBJ Closing and VNBZ Closing, respectively. Veoneer Parties cease to be a party to the JV Agreement and have no further obligations, while the agreement remains in effect between Nissin Parties and Autoliv AB with respect to the liquidation proceedings of ANRA. The termination does not release a

In [10]:
def get_answer(input: str) -> str:
    input_messages = [HumanMessage(content=input)]
    messages = agent.invoke({"messages": input_messages})
    tools = [m.additional_kwargs.get('function_call') if m.additional_kwargs.get('function_call') else  m.additional_kwargs.get("tool_calls") for m in messages["messages"] if isinstance(m, AIMessage) and (m.additional_kwargs.get('function_call') or m.additional_kwargs.get('tool_calls'))]
    context = [m.content for m in messages["messages"] if isinstance(m, ToolMessage)]
    answer = messages["messages"][-1].content
    return answer, tools, context

In [11]:
from tqdm import tqdm

# Get workflows generated answers
results = []
latencies = []
for i, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing questions"):
    question = row['question']
    start = time.time()
    try:
        data = get_answer(question)
    except:
        data = {"answer": "timeout", "question": question}
    end = time.time()
    latencies.append(end - start)
    results.append(data)

test_df["latencies"] = latencies
test_df["response"] = [el[0] for el in results]
test_df["tools"] = [el[1] for el in results]
test_df["retrieved_contexts"] = [[str(item) for item in el[2]] for el in results]

Processing questions: 100%|██████████| 44/44 [03:25<00:00,  4.67s/it]


In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)  # Adjust width to prevent wrapping
pd.set_option('display.max_colwidth', None)  # Prevents truncation of long column values

In [13]:
test_df[['response','question', 'tools']]

,response,question,tools
0,"I found several contracts that ended after January 1, 1990. Here are some examples:\n\n1. **Strategic Alliance Agreement**\n - **Parties Involved:** PHL Variable Insurance Company, Phoenix Life Insurance Company, Phoenix Equity Planning Corporation, and Investors Capital Corporation (ICC).\n - **End Date:** December 6, 2033\n - **Type:** Strategic Alliance\n - **Summary:** This agreement involves collaboration among insurance and financial entities for asset allocation models and sales solicitation. \n\n2. **Service Agreement**\n - **Parties Involved:** Rise (Tianjin) Education Information Consulting Co., Ltd. and an anonymous service recipient.\n - **End Date:** August 24, 2028\n - **Type:** Service\n - **Summary:** Operation support services are provided, including academic, financial, and legal support.\n\n3. **Consulting Agreement**\n - **Parties Involved:** Aduro Biotech, Inc. and IREYA B.V.\n - **End Date:** December 31, 2020\n - **Type:** Service\n - **Summary:** Aduro Biotech receives consultation services regarding research and development programs.\n\n4. **Investment Advisory Agreement**\n - **Parties Involved:** PFS Funds and Potomac Fund Management, Inc.\n - **End Date:** June 24, 2022\n - **Type:** Service\n - **Summary:** Potomac Fund Management acts as an investment adviser for several funds under PFS Funds.\n\nThese examples demonstrate the range of contract types and services that companies engage in, extending into various areas such as strategic alliances, service provision, and consultation.",What are some contracts that ended after 1990-01-01?,"[[{'id': 'call_hP1zbFdPasjmajWaOLyoUxvP', 'function': {'arguments': '{""min_end_date"":""1990-01-01""}', 'name': 'ContractSearch'}, 'type': 'function'}]]"
1,"There are several strategic alliance contracts that ended after January 1, 1990. Here are some notable examples:\n\n1. **PHL Variable Insurance Company et al.** (End Date: December 6, 2033)\n - **Parties Involved:** PHL VARIABLE INSURANCE COMPANY, PHOENIX LIFE INSURANCE COMPANY, PHOENIX EQUITY PLANNING CORPORATION, INVESTORS CAPITAL CORPORATION\n - **Summary:** This agreement involves collaboration on asset allocation models and the sale of Master Group Annuity Contracts.\n - **Effective Date:** December 6, 2023\n\n2. **Spôk Holdings, Inc. & White Hat Strategic Partners** (End Date: May 18, 2021)\n - **Parties Involved:** Spôk Holdings, Inc. and White Hat Strategic Partners LP, among others\n - **Summary:** The agreement covers board appointments and stockholder voting arrangements.\n - **Monetary Value:** $55,000\n - **Effective Date:** June 18, 2020\n\n3. **HC2 Holdings, Inc. & MG Capital Management Ltd.** (End Date: January 1, 2021)\n - **Parties Involved:** MG Capital Management Ltd. and HC2 Holdings, Inc.\n - **Summary:** This agreement modifies the company's board composition and includes provisions related to stockholder meetings.\n - **Effective Date:** May 13, 2020\n\n4. **Liquidmetal Technologies, Inc. & Eutectix, LLC** (End Date: January 1, 2025)\n - **Parties Involved:** Liquidmetal Technologies, Inc. and Eutectix, LLC\n - **Summary:** Collaboration for alloy development and manufacturing, including royalties on licensed products.\n - **Effective Date:** January 1, 2020\n\n5. **Farids & Co. LLC, Edible Arrangements, LLC & Rocky Mountain Chocolate Factory, Inc.** (End Date: December 20, 2019)\n - **Parties Involved:** Farids & Co. LLC, Edible Arrangements, LLC, Rocky Mountain Chocolate Factory, Inc.\n - **Summary:** Enhancing strategic cooperation and issuing common stock shares.\n - **Effective Date:** December 20, 2019\n\nThese contracts highlight various aspects of strategic alliances, such as board cooperation, asset management collaborations, and business development initiatives.",What are some contracts that ended after 1990-01-01 that fall under strategic alliance?,"[[{'id': 'call_DpyN5UlB2TJKl8SiWTRWiQFz', 'function': {'arguments': '{""min_end_date"":""1990-01-01""

In [14]:
from ragas.metrics import AspectCritic

# you can init the metric with the evaluator llm
answer_satisfaction = AspectCritic(
    name="answer_satisfaction",
    definition="""You will evaluate an ANSWER to a legal QUESTION based on a provided SOLUTION.

Rate the answer on a scale from 0 to 1, where:
- 0 = incorrect, substantially incomplete, or misleading
- 1 = correct and sufficiently complete

Consider these evaluation criteria:
1. Factual correctness is paramount - the answer must not contradict the solution
2. The answer must address the core elements of the solution
3. For complex legal questions, the answer should capture the essential legal principles even if not every detail is included
4. Additional relevant information beyond the solution is acceptable and may enhance the answer
5. Technical legal terminology should be used appropriately if present in the solution
6. For quantitative legal analyses, accurate figures must be provided

### Examples:

QUESTION: What is the statute of limitations for breach of contract in California?
GROUND_TRUTH: In California, the statute of limitations for written contracts is 4 years, and for oral contracts is 2 years, as specified in California Code of Civil Procedure § 337 and § 339.
RESPONSE: California has a 4-year statute of limitations for written contract claims and 2 years for oral contracts.
THOUGHTS: The answer correctly identifies both time periods (4 years for written contracts and 2 years for oral contracts) in California. While it doesn't cite the specific code sections, it accurately conveys the essential legal information from the solution.
RATING: 1

QUESTION: What is required for a valid contract?
GROUND_TRUTH: A valid contract requires offer, acceptance, consideration, legal capacity of the parties, legal purpose, and mutual assent (meeting of the minds).
RESPONSE: For a contract to be valid, there must be consideration exchanged between the parties.
RATING: 0

QUESTION: How does the Fourth Amendment protect citizens?
GROUND_TRUTH: The Fourth Amendment protects citizens from unreasonable searches and seizures by the government, requiring probable cause for warrants, which must specify the place to be searched and the persons or things to be seized.
RESPONSE: The Fourth Amendment prohibits unreasonable searches and seizures, requires warrants based on probable cause, and these warrants must describe specifically what and where officers are searching.
RATING: 1""",
)

In [15]:
dataset = Dataset.from_pandas(test_df)

result = evaluate(
    dataset,
    metrics=[
        answer_correctness,
        context_recall,
        faithfulness,
        answer_satisfaction
    ],
)

Evaluating:   0%|          | 0/176 [00:00<?, ?it/s]

Exception raised in Job[17]: KeyError('reference')
Exception raised in Job[16]: KeyError('reference')
Exception raised in Job[29]: KeyError('reference')
Exception raised in Job[36]: KeyError('reference')
Exception raised in Job[37]: KeyError('reference')
Exception raised in Job[28]: KeyError('reference')
Exception raised in Job[41]: KeyError('reference')
Exception raised in Job[45]: KeyError('reference')
Exception raised in Job[40]: KeyError('reference')
Exception raised in Job[53]: KeyError('reference')
Exception raised in Job[52]: KeyError('reference')
Exception raised in Job[61]: KeyError('reference')
Exception raised in Job[65]: KeyError('reference')
Exception raised in Job[60]: KeyError('reference')
Exception raised in Job[64]: KeyError('reference')
Exception raised in Job[69]: KeyError('reference')
Exception raised in Job[68]: KeyError('reference')
Exception raised in Job[77]: KeyError('reference')
Exception raised in Job[44]: KeyError('reference')
Exception raised in Job[81]: Ke

In [17]:
result

{'answer_correctness': 0.6192, 'context_recall': 0.8889, 'faithfulness': 0.6026, 'answer_satisfaction': 0.6364}

In [16]:
result.to_pandas()[['user_input','answer_correctness', 'context_recall', 'faithfulness', 'answer_satisfaction', 'response', 'reference']]

,user_input,answer_correctness,context_recall,faithfulness,answer_satisfaction,response,reference
0,What are some contracts that ended after 1990-01-01?,0.274520,1.0,0.739130,1,"I found several contracts that ended after January 1, 1990. Here are some examples:\n\n1. **Strategic Alliance Agreement**\n - **Parties Involved:** PHL Variable Insurance Company, Phoenix Life Insurance Company, Phoenix Equity Planning Corporation, and Investors Capital Corporation (ICC).\n - **End Date:** December 6, 2033\n - **Type:** Strategic Alliance\n - **Summary:** This agreement involves collaboration among insurance and financial entities for asset allocation models and sales solicitation. \n\n2. **Service Agreement**\n - **Parties Involved:** Rise (Tianjin) Education Information Consulting Co., Ltd. and an anonymous service recipient.\n - **End Date:** August 24, 2028\n - **Type:** Service\n - **Summary:** Operation support services are provided, including academic, financial, and legal support.\n\n3. **Consulting Agreement**\n - **Parties Involved:** Aduro Biotech, Inc. and IREYA B.V.\n - **End Date:** December 31, 2020\n - **Type:** Service\n - **Summary:** Aduro Biotech receives consultation services regarding research and development programs.\n\n4. **Investment Advisory Agreement**\n - **Parties Involved:** PFS Funds and Potomac Fund Management, Inc.\n - **End Date:** June 24, 2022\n - **Type:** Service\n - **Summary:** Potomac Fund Management acts as an investment adviser for several funds under PFS Funds.\n\nThese examples demonstrate the range of contract types and services that companies engage in, extending into various areas such as strategic alliances, service provision, and consultation.",I found 323 contracts that ended after 1990-01-01.
1,What are some contracts that ended after 1990-01-01 that fall under strategic alliance?,0.339449,1.0,0.956522,1,"There are several strategic alliance contracts that ended after January 1, 1990. Here are some notable examples:\n\n1. **PHL Variable Insurance Company et al.** (End Date: December 6, 2033)\n - **Parties Involved:** PHL VARIABLE INSURANCE COMPANY, PHOENIX LIFE INSURANCE COMPANY, PHOENIX EQUITY PLANNING CORPORATION, INVESTORS CAPITAL CORPORATION\n - **Summary:** This agreement involves collaboration on asset allocation models and the sale of Master Group Annuity Contracts.\n - **Effective Date:** December 6, 2023\n\n2. **Spôk Holdings, Inc. & White Hat Strategic Partners** (End Date: May 18, 2021)\n - **Parties Involved:** Spôk Holdings, Inc. and White Hat Strategic Partners LP, among others\n - **Summary:** The agreement covers board appointments and stockholder voting arrangements.\n - **Monetary Value:** $55,000\n - **Effective Date:** June 18, 2020\n\n3. **HC2 Holdings, Inc. & MG Capital Management Ltd.** (End Date: January 1, 2021)\n - **Parties Involved:** MG Capital Management Ltd. and HC2 Holdings, Inc.\n - **Summary:** This agreement modifies the company's board composition and includes provisions related to stockholder meetings.\n - **Effective Date:** May 13, 2020\n\n4. **Liquidmetal Technologies, Inc. & Eutectix, LLC** (End Date: January 1, 2025)\n - **Parties Involved:** Liquidmetal Technologies, Inc. and Eutectix, LLC\n - **Summary:** Collaboration for alloy development and manufacturing, including royalties on licensed products.\n - **Effective Date:** January 1, 2020\n\n5. **Farids & Co. LLC, Edible Arrangements, LLC & Rocky Mountain Chocolate Factory, Inc.** (End Date: December 20, 2019)\n - **Parties Involved:** Farids & Co. LLC, Edible Arrangements, LLC, Rocky Mountain Chocolate Factory, Inc.\n - **Summary:** Enhancing strategic cooperation and issuing common stock shares.\n - **Effective Date:** December 20, 2019\n\nThese contracts highlight various aspects of strategic alliances, such as board cooperation, asset management collaborations, and business development initiatives.","There are 52 contracts classified as ""Strategic Alliance"" that ended after January 